In [2]:
from src import db, processor, model, loader, generator

In [ ]:
print('🟢Starting ETL process')

# Define the file path for the raw data
file_path = "data/raw/Invoices_Year_2009-2010.csv"

# Clean and process the raw data
df = processor.process_data(file_path)

In [ ]:
# Create the table invoices in PostgreSQL
db.upsert_table(df, "invoices")

In [ ]:
# Execute a base SQL query and get the data to provide to the model
df_clean = db.execute_sql_file("src/utils/queries/base_query.sql")

In [ ]:
# Generate the data model from the data (Kimball model)
dim_product, dim_customer, dim_date, dim_country, fact_sales = model.generate_model(
    df_clean
)

In [ ]:
# Upload the generated data model to PostgreSQL
loader.upload_model(dim_product, dim_customer, dim_date, dim_country, fact_sales)

print('🟢ETL process completed')

In [ ]:
print("🕒Now the aggregations of data, detailed comments on each file of queries")
# Aggregations 
df_agg_1 = db.execute_sql_file("src/utils/queries/growth_yoy_by_country.sql")
print("First aggregation of data")
print(df_agg_1)
df_agg_2 = db.execute_sql_file("src/utils/queries/cohort_analysis.sql")
print("Second aggregation of data")
print(df_agg_2)
df_agg_3 = db.execute_sql_file("src/utils/queries/classification_products.sql")
print("Third aggregation of data")
print(df_agg_3)

In [ ]:
print("⚠️Now try the generator with some queries, the generator use the data from the database")
# BETA MODE: Generate the table from input of the user
# use for simple queries
# try: "top 10 products sold in December 2009"

user_query = "top 10 products sold in December 2009"
df_generated = generator.generate_table(user_query)
print(df_generated)

In [3]:
print(db.execute_query("select table_name from information_schema.tables where table_schema='public'"))

🔒 Connection closed
     table_name
0      dim_date
1   dim_country
2    test_table
3      invoices
4    fact_sales
5   dim_product
6  dim_customer


In [4]:
db.execute_query("DROP TABLE invoices")
db.execute_query("DROP TABLE dim_product")
db.execute_query("DROP TABLE dim_customer")
db.execute_query("DROP TABLE dim_date")
db.execute_query("DROP TABLE dim_country")
db.execute_query("DROP TABLE fact_sales")
db.execute_query("DROP TABLE test_table")

✅ Query executed successfully
🔒 Connection closed
✅ Query executed successfully
🔒 Connection closed
✅ Query executed successfully
🔒 Connection closed
✅ Query executed successfully
🔒 Connection closed
✅ Query executed successfully
🔒 Connection closed
✅ Query executed successfully
🔒 Connection closed
✅ Query executed successfully
🔒 Connection closed
